In [1]:
import pandas as pd
import numpy as np
import openpyxl

营养素的库存管理复杂，经手人多，账套混乱，操作原始。核对起来相当痛苦。

为了做一个全局管理脚本，在这理一下思路。

首先，营养素有住院部和服务部2个部门在同时售卖。
住院部的库存在食堂仓库，需要人工清点。服务部的库存在服务部，需要人工清点。

首先，营养素的管理有两个系统。
## 总系统：百威进销存系统
百威进销存系统记录库存，这个系统减去当天服务部售卖的营养素，但是住院部售卖的营养素需要到月底根据手工统计减去。
### 医院的记账系统 
医院的记账系统只记录住院部的售卖情况。不记录库存。
住院部的工作除我以外还有两人参与：

住院部订单员：下达订单，通知仓库管理员来取货。自己维护一张订单表格

仓库管理员：接到通知时在当日内取货。并计入医院的记账系统
#### 出错可能
食堂管理员少拿错拿营养素，记错帐
#### 开发功能需求
定期核对前一日食堂管理员的医院帐和订单员维护的帐
### 服务部的记账方法 
服务部的帐月末结入百威系统，当月售卖
#### 出错可能
服务部在系统内记账，记漏记错
#### 开发功能需求
记录服务部每日库存差和挂账是否相等

#### 综合需求
昨日百威进销存系统的结存数量 = 仓库库存+服务部库存+服务部挂账+核对后的住院部订单
## 库存管理：

### 入库：
入库由我报数量下订单给采购员，采购员送货后。服务部将其录入。由于服务部有退货可能，所以需在服务部验收后将数量报给我。
### 借调
服务部库存不够时，由我向食堂调货。这时，需在系统内调整库存。
## 保质期管理：
没想好怎么管，各种批号的营养素掺在一起 麻烦炸了先记录下来 功能保留 （用概率论猜？）
## 回溯查询：
要有每次查询要有历史记录，可以随时回档。使用git进行版本管理
## 使用场景
每天早上
我：核对订单员和医院记账系统是否对应

系统：读取订单员表格 数据清洗整理 统计出每天每个品卖出几个 去记账系统看

我：准备一张表，填入核对日期，入库量，失效日期，百威结存数量，仓库库存，服务部库存（允许填不变），昨日借调数量，服务部挂账，住院部售卖数量

系统：核对数据，提示出错商品错了几个。并更新明天的表格

我：git add git commit git push



In [2]:
df = pd.read_csv('inventory_count.csv')
df

,RQDB,1,0
0,YZS,2,1
1,YTS,3,1
2,YNS,4,1
3,YQS,4,1
4,YDS,6,2
5,YJS,7,1
6,YMZ,8,1
7,YMS,9,1


清洗数据

In [5]:
zhuyuandata = pd.read_excel('yys.xls',header=1,sheet_name=1)
zhuyuandata =zhuyuandata.iloc[:len(zhuyuandata)-1,]
zhuyuandata = zhuyuandata.dropna(axis=0,how='all')
zhuyuandata.head()

,日期,病区,床号,姓名,住院号,出生年月,营养制剂名称,数量,费用,配送人员
0,9.01,B3X,3.0,夏香凤,380958.0,1943-07-08 00:00:00,乳清蛋白,1.0,188,黄海琴
1,,B3X,19.0,彭碧华,408027.0,1953-12-16 00:00:00,乳清蛋白,1.0,188,黄海琴
2,9.02,B3X,21.0,袁其红,242889.0,1953-03-03 00:00:00,乳清蛋白,1.0,188,王艳/朱
3,9.03,A6D,33.0,郑荷香,357954.0,1952/6/27,乳清蛋白,1.0,188,王艳/朱
4,NaN,B3X,18.0,张邦才,404135.0,1956-09-21 00:00:00,乳清蛋白,1.0,188,王艳/朱


将完全为空的格子转换为NaN

In [4]:
a = (zhuyuandata.isnull()|zhuyuandata.applymap(lambda x:str(x).isspace()))
#zhuyuandata_null = zhuyuandata[a]
#huyuandata_null
zhuyuandata_not_null =zhuyuandata[~a]
#删除两边的空格
zhuyuandata_not_null=zhuyuandata_not_null.applymap(lambda x:str(x).strip('s'))
zhuyuandata_not_null.head()


,日期,病区,床号,姓名,住院号,出生年月,营养制剂名称,数量,费用,配送人员
0,9.01,B3X,3.0,夏香凤,380958.0,1943-07-08 00:00:00,乳清蛋白,1.0,188,黄海琴
1,nan,B3X,19.0,彭碧华,408027.0,1953-12-16 00:00:00,乳清蛋白,1.0,188,黄海琴
2,9.02,B3X,21.0,袁其红,242889.0,1953-03-03 00:00:00,乳清蛋白,1.0,188,王艳/朱
3,9.03,A6D,33.0,郑荷香,357954.0,1952/6/27,乳清蛋白,1.0,188,王艳/朱
4,nan,B3X,18.0,张邦才,404135.0,1956-09-21 00:00:00,乳清蛋白,1.0,188,王艳/朱


In [16]:
for i in zhuyuandata_not_null["日期"]:
    if i != "nan":
        print ('aa')

aa
aa
aa
aa
aa
aa
aa
aa
aa
aa
aa
aa
aa
aa
aa
aa
aa
aa
aa
aa
aa
aa


In [22]:
for i in zhuyuandata_not_null["日期"]:
    if i == "nan":
        zhuyuandata_not_null['日期'].at[i]= '短短的'
zhuyuandata_not_null

,日期,病区,床号,姓名,住院号,出生年月,营养制剂名称,数量,费用,配送人员
0,9.01,B3X,3.0,夏香凤,380958.0,1943-07-08 00:00:00,乳清蛋白,1.0,188,黄海琴
1,nan,B3X,19.0,彭碧华,408027.0,1953-12-16 00:00:00,乳清蛋白,1.0,188,黄海琴
2,9.02,B3X,21.0,袁其红,242889.0,1953-03-03 00:00:00,乳清蛋白,1.0,188,王艳/朱
3,9.03,A6D,33.0,郑荷香,357954.0,1952/6/27,乳清蛋白,1.0,188,王艳/朱
4,nan,B3X,18.0,张邦才,404135.0,1956-09-21 00:00:00,乳清蛋白,1.0,188,王艳/朱
5,nan,nan,nan,nan,nan,nan,伊能素,1.0,380,王艳/朱
6,nan,B7D,16.0,陈夏领,408407.0,1933-05-07 00:00:00,乳清蛋白,1.0,188,王艳/朱
7,nan,nan,nan,nan,nan,nan,伊能素,1.0,380,王艳/朱
8,9.05,A6D,36.0,林定生,408754.0,1942-01-19 00:00:00,乳清蛋白,1.0,188,王艳/朱
9,nan,nan,nan,nan,nan,nan,伊全素,1.0,98,王艳/朱
